# Sales Data Generator for Workshop

This notebook generates realistic sales data for the **Fabric for the Power BI User** workshop.

**Run this in:** Microsoft Fabric Lakehouse environment

**What it creates:**
- `Products` - 65 outdoor/sporting goods products
- `Stores` - 12 retail locations across Pacific Northwest
- `Calendar` - Date dimension (2023-2025)
- `Targets` - Monthly revenue targets by store
- `Sales` - Transaction-level sales data

---

## Configuration

Adjust these settings as needed for your demo:

In [ ]:
# Configuration
NUM_TRANSACTIONS = 100000  # Number of sales transactions to generate
START_DATE = "2023-01-01"
END_DATE = "2025-12-31"

# Set to True to append new data (for incremental demo)
APPEND_MODE = False

# Number of new transactions to add in append mode
APPEND_COUNT = 5000

## Setup & Imports

In [ ]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Spark session (available by default in Fabric)
spark = SparkSession.builder.getOrCreate()

print("Spark session ready!")

## 1. Products Table

65 products across 8 categories of outdoor and sporting goods.

In [ ]:
products_data = [
    ("1001", "Trail Runner Pro", "Footwear", "Running Shoes", 129.99, 52.00, True),
    ("1002", "Summit Hiker", "Footwear", "Hiking Boots", 189.99, 76.00, True),
    ("1003", "Urban Walker", "Footwear", "Casual Shoes", 79.99, 32.00, True),
    ("1004", "Grip Master Sandal", "Footwear", "Sandals", 59.99, 24.00, True),
    ("1005", "Winter Trek Boot", "Footwear", "Winter Boots", 219.99, 88.00, True),
    ("1006", "Sprint Elite", "Footwear", "Running Shoes", 149.99, 60.00, True),
    ("1007", "All-Terrain Mid", "Footwear", "Hiking Boots", 159.99, 64.00, True),
    ("1008", "Classic Canvas", "Footwear", "Casual Shoes", 49.99, 20.00, True),
    ("2001", "Ultralight Tent 2P", "Camping", "Tents", 349.99, 140.00, True),
    ("2002", "Family Dome 6P", "Camping", "Tents", 449.99, 180.00, True),
    ("2003", "Compact Sleep Bag", "Camping", "Sleeping Bags", 129.99, 52.00, True),
    ("2004", "Arctic Sleep Bag", "Camping", "Sleeping Bags", 249.99, 100.00, True),
    ("2005", "Quick Setup Canopy", "Camping", "Shelters", 199.99, 80.00, True),
    ("2006", "Trekking Poles Set", "Camping", "Accessories", 79.99, 32.00, True),
    ("2007", "LED Headlamp Pro", "Camping", "Lighting", 44.99, 18.00, True),
    ("2008", "Camp Chair Deluxe", "Camping", "Furniture", 89.99, 36.00, True),
    ("2009", "Portable Stove", "Camping", "Cooking", 69.99, 28.00, True),
    ("2010", "Water Filter System", "Camping", "Hydration", 49.99, 20.00, True),
    ("3001", "Performance Tee", "Apparel", "Tops", 39.99, 16.00, True),
    ("3002", "Fleece Pullover", "Apparel", "Tops", 79.99, 32.00, True),
    ("3003", "Rain Shell Jacket", "Apparel", "Outerwear", 149.99, 60.00, True),
    ("3004", "Down Insulated Jacket", "Apparel", "Outerwear", 229.99, 92.00, True),
    ("3005", "Hiking Pants", "Apparel", "Bottoms", 69.99, 28.00, True),
    ("3006", "Quick-Dry Shorts", "Apparel", "Bottoms", 49.99, 20.00, True),
    ("3007", "Merino Base Layer", "Apparel", "Base Layers", 89.99, 36.00, True),
    ("3008", "Sport Socks 3-Pack", "Apparel", "Accessories", 24.99, 10.00, True),
    ("3009", "Sun Hat Wide Brim", "Apparel", "Accessories", 34.99, 14.00, True),
    ("3010", "Winter Beanie", "Apparel", "Accessories", 29.99, 12.00, True),
    ("4001", "Mountain Bike 27.5", "Cycling", "Bikes", 899.99, 450.00, True),
    ("4002", "Road Bike Carbon", "Cycling", "Bikes", 1499.99, 750.00, True),
    ("4003", "Kids Bike 20in", "Cycling", "Bikes", 299.99, 150.00, True),
    ("4004", "Bike Helmet Adult", "Cycling", "Safety", 79.99, 32.00, True),
    ("4005", "Bike Lock Heavy Duty", "Cycling", "Accessories", 49.99, 20.00, True),
    ("4006", "Cycling Gloves", "Cycling", "Apparel", 34.99, 14.00, True),
    ("4007", "Bike Repair Kit", "Cycling", "Tools", 29.99, 12.00, True),
    ("4008", "Water Bottle Cage", "Cycling", "Accessories", 14.99, 6.00, True),
    ("5001", "Kayak Single 10ft", "Water Sports", "Kayaks", 649.99, 260.00, True),
    ("5002", "Kayak Tandem 12ft", "Water Sports", "Kayaks", 899.99, 360.00, True),
    ("5003", "Paddle Adjustable", "Water Sports", "Paddles", 89.99, 36.00, True),
    ("5004", "Life Vest Adult", "Water Sports", "Safety", 69.99, 28.00, True),
    ("5005", "Dry Bag 20L", "Water Sports", "Storage", 39.99, 16.00, True),
    ("5006", "Snorkel Set", "Water Sports", "Snorkeling", 49.99, 20.00, True),
    ("5007", "Beach Towel XL", "Water Sports", "Accessories", 29.99, 12.00, True),
    ("6001", "Yoga Mat Premium", "Fitness", "Yoga", 49.99, 20.00, True),
    ("6002", "Resistance Bands Set", "Fitness", "Strength", 29.99, 12.00, True),
    ("6003", "Dumbbell Set 50lb", "Fitness", "Weights", 149.99, 60.00, True),
    ("6004", "Kettlebell 25lb", "Fitness", "Weights", 59.99, 24.00, True),
    ("6005", "Jump Rope Speed", "Fitness", "Cardio", 19.99, 8.00, True),
    ("6006", "Exercise Ball 65cm", "Fitness", "Accessories", 34.99, 14.00, True),
    ("6007", "Foam Roller", "Fitness", "Recovery", 29.99, 12.00, True),
    ("6008", "Fitness Tracker Band", "Fitness", "Electronics", 99.99, 40.00, True),
    ("7001", "Ski Package Adult", "Winter Sports", "Skiing", 599.99, 240.00, True),
    ("7002", "Snowboard Complete", "Winter Sports", "Snowboarding", 449.99, 180.00, True),
    ("7003", "Ski Goggles Pro", "Winter Sports", "Accessories", 89.99, 36.00, True),
    ("7004", "Snow Gloves Insulated", "Winter Sports", "Apparel", 59.99, 24.00, True),
    ("7005", "Ski Poles Aluminum", "Winter Sports", "Skiing", 49.99, 20.00, True),
    ("7006", "Snowshoes Adult", "Winter Sports", "Snowshoeing", 179.99, 72.00, True),
    ("7007", "Hand Warmers 10-Pack", "Winter Sports", "Accessories", 14.99, 6.00, True),
    ("8001", "Backpack 40L", "Bags", "Hiking Packs", 129.99, 52.00, True),
    ("8002", "Backpack 65L", "Bags", "Hiking Packs", 179.99, 72.00, True),
    ("8003", "Daypack 25L", "Bags", "Daypacks", 79.99, 32.00, True),
    ("8004", "Hydration Pack", "Bags", "Hydration", 99.99, 40.00, True),
    ("8005", "Duffel Bag Large", "Bags", "Travel", 89.99, 36.00, True),
    ("8006", "Waist Pack", "Bags", "Accessories", 34.99, 14.00, True),
    ("8007", "Laptop Backpack", "Bags", "Urban", 109.99, 44.00, True),
    ("8008", "Cooler Bag 24-Can", "Bags", "Coolers", 49.99, 20.00, True),
]

products_schema = StructType([
    StructField("ProductID", StringType(), False),
    StructField("ProductName", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("SubCategory", StringType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("UnitCost", DoubleType(), True),
    StructField("IsActive", BooleanType(), True),
])

df_products = spark.createDataFrame(products_data, products_schema)
print(f"Products: {df_products.count()} rows")
df_products.show(5)

## 2. Stores Table

12 retail locations across Pacific Northwest and Mountain regions.

In [ ]:
stores_data = [
    ("S001", "Downtown Flagship", "Seattle", "WA", "Pacific Northwest", "2018-03-15", 15000, "Sarah Mitchell"),
    ("S002", "Eastside Mall", "Bellevue", "WA", "Pacific Northwest", "2019-06-01", 8500, "James Chen"),
    ("S003", "University District", "Seattle", "WA", "Pacific Northwest", "2020-02-20", 6000, "Maria Rodriguez"),
    ("S004", "Tacoma Central", "Tacoma", "WA", "Pacific Northwest", "2019-11-10", 7500, "David Kim"),
    ("S005", "Portland Pearl", "Portland", "OR", "Pacific Northwest", "2018-08-22", 12000, "Emily Watson"),
    ("S006", "Portland East", "Gresham", "OR", "Pacific Northwest", "2021-04-15", 6500, "Michael Brown"),
    ("S007", "Eugene Campus", "Eugene", "OR", "Pacific Northwest", "2020-09-01", 5500, "Lisa Park"),
    ("S008", "Boise Main", "Boise", "ID", "Mountain", "2019-03-10", 9000, "Robert Taylor"),
    ("S009", "Boise West", "Meridian", "ID", "Mountain", "2022-01-20", 5000, "Jennifer Adams"),
    ("S010", "Spokane Valley", "Spokane", "WA", "Mountain", "2020-06-15", 7000, "Chris Martinez"),
    ("S011", "Missoula Downtown", "Missoula", "MT", "Mountain", "2021-08-01", 5500, "Amanda Wilson"),
    ("S012", "Bend Outdoor", "Bend", "OR", "Pacific Northwest", "2022-05-15", 6000, "Kevin O'Brien"),
]

stores_schema = StructType([
    StructField("StoreID", StringType(), False),
    StructField("StoreName", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("OpenDate", StringType(), True),
    StructField("SquareFeet", IntegerType(), True),
    StructField("ManagerName", StringType(), True),
])

df_stores = spark.createDataFrame(stores_data, stores_schema)
df_stores = df_stores.withColumn("OpenDate", to_date(col("OpenDate")))
print(f"Stores: {df_stores.count()} rows")
df_stores.show()

## 3. Calendar Table

Date dimension covering 2023-2025 with fiscal calendar support.

In [ ]:
from pyspark.sql.functions import explode, sequence, to_date, year, quarter, month, dayofweek, weekofyear
from pyspark.sql.functions import date_format, when, lit

# Generate date range
df_calendar = spark.sql(f"""
    SELECT explode(sequence(to_date('{START_DATE}'), to_date('{END_DATE}'), interval 1 day)) as Date
""")

# Add calendar columns
df_calendar = df_calendar \
    .withColumn("Year", year("Date")) \
    .withColumn("Quarter", concat(lit("Q"), quarter("Date"))) \
    .withColumn("Month", month("Date")) \
    .withColumn("MonthName", date_format("Date", "MMMM")) \
    .withColumn("WeekOfYear", weekofyear("Date")) \
    .withColumn("DayOfWeek", dayofweek("Date")) \
    .withColumn("DayName", date_format("Date", "EEEE")) \
    .withColumn("IsWeekend", when(dayofweek("Date").isin([1, 7]), True).otherwise(False)) \
    .withColumn("IsHoliday", lit(False)) \
    .withColumn("FiscalYear", 
        when(month("Date") >= 7, concat(lit("FY"), year("Date") + 1))
        .otherwise(concat(lit("FY"), year("Date")))) \
    .withColumn("FiscalQuarter",
        when(month("Date").isin([7,8,9]), lit("FQ1"))
        .when(month("Date").isin([10,11,12]), lit("FQ2"))
        .when(month("Date").isin([1,2,3]), lit("FQ3"))
        .otherwise(lit("FQ4")))

# Mark US holidays (simplified)
holidays = [
    "2023-01-01", "2023-01-16", "2023-05-29", "2023-07-04", "2023-09-04", 
    "2023-11-23", "2023-11-24", "2023-12-25",
    "2024-01-01", "2024-01-15", "2024-05-27", "2024-07-04", "2024-09-02",
    "2024-11-28", "2024-11-29", "2024-12-25",
    "2025-01-01", "2025-01-20", "2025-05-26", "2025-07-04", "2025-09-01",
    "2025-11-27", "2025-11-28", "2025-12-25",
]
df_calendar = df_calendar.withColumn("IsHoliday", 
    when(col("Date").isin([to_date(lit(h)) for h in holidays]), True).otherwise(False))

print(f"Calendar: {df_calendar.count()} rows")
df_calendar.show(5)

## 4. Targets Table

Monthly revenue and transaction targets by store for 2024.

In [ ]:
# Build targets using Spark SQL to avoid any type/function conflicts

# Create temporary view with store data
store_targets_sql = """
SELECT StoreID, AnnualTarget FROM (VALUES
    ('S001', 2700000),
    ('S002', 1450000),
    ('S003', 975000),
    ('S004', 1168000),
    ('S005', 2176000),
    ('S006', 1028000),
    ('S007', 907000),
    ('S008', 1480000),
    ('S009', 794000),
    ('S010', 1123000),
    ('S011', 906000),
    ('S012', 1028000)
) AS t(StoreID, AnnualTarget)
"""

# Create temporary view with monthly weights
monthly_sql = """
SELECT MonthNum, Weight FROM (VALUES
    (1, 0.07), (2, 0.065), (3, 0.075), (4, 0.085),
    (5, 0.095), (6, 0.10), (7, 0.11), (8, 0.105),
    (9, 0.09), (10, 0.08), (11, 0.10), (12, 0.12)
) AS t(MonthNum, Weight)
"""

# Generate targets using pure SQL
df_targets = spark.sql(f"""
WITH stores AS ({store_targets_sql}),
     months AS ({monthly_sql})
SELECT 
    s.StoreID,
    2024 AS Year,
    m.MonthNum AS Month,
    CAST(ROUND(s.AnnualTarget * m.Weight, -2) AS DOUBLE) AS RevenueTarget,
    CAST(ROUND(s.AnnualTarget * m.Weight, -2) / 100 AS INT) AS TransactionTarget
FROM stores s
CROSS JOIN months m
ORDER BY s.StoreID, m.MonthNum
""")

print(f"Targets: {df_targets.count()} rows")
df_targets.show(5)

## 5. Sales Transactions

Generate realistic transaction data with seasonality and store weighting.

In [ ]:
import random
from datetime import datetime, timedelta

# Seasonality factors by month (outdoor retail patterns)
SEASONALITY = {
    1: 0.7, 2: 0.65, 3: 0.85, 4: 1.0, 5: 1.2, 6: 1.3,
    7: 1.35, 8: 1.25, 9: 0.95, 10: 0.9, 11: 1.1, 12: 1.4
}

# Store size weights
STORE_WEIGHTS = {
    "S001": 2.0, "S002": 1.2, "S003": 0.8, "S004": 1.0,
    "S005": 1.6, "S006": 0.9, "S007": 0.75, "S008": 1.3,
    "S009": 0.7, "S010": 1.0, "S011": 0.75, "S012": 0.85
}

# Store open dates
STORE_OPEN = {
    "S001": datetime(2018, 3, 15), "S002": datetime(2019, 6, 1),
    "S003": datetime(2020, 2, 20), "S004": datetime(2019, 11, 10),
    "S005": datetime(2018, 8, 22), "S006": datetime(2021, 4, 15),
    "S007": datetime(2020, 9, 1), "S008": datetime(2019, 3, 10),
    "S009": datetime(2022, 1, 20), "S010": datetime(2020, 6, 15),
    "S011": datetime(2021, 8, 1), "S012": datetime(2022, 5, 15)
}

# Collect products for local use
products_list = df_products.collect()
store_ids = list(STORE_WEIGHTS.keys())

# Helper to safely round to 2 decimal places as float
def round2(val):
    return float(int(val * 100 + 0.5) / 100)

def generate_transactions(num_transactions, start_id=1000001):
    """Generate sales transaction records"""
    transactions = []
    start = datetime.strptime(START_DATE, "%Y-%m-%d")
    end = datetime.strptime(END_DATE, "%Y-%m-%d")
    total_days = (end - start).days
    
    for i in range(num_transactions):
        txn_id = f"T{start_id + i}"
        
        # Random date
        days_offset = random.randint(0, total_days)
        txn_date = start + timedelta(days=days_offset)
        
        # Select store (weighted)
        store_id = random.choices(
            store_ids,
            weights=[STORE_WEIGHTS[s] for s in store_ids]
        )[0]
        
        # Skip if store wasn't open
        if txn_date < STORE_OPEN[store_id]:
            continue
        
        # Select product
        product = random.choice(products_list)
        
        # Quantity
        qty = 1 if random.random() < 0.8 else random.randint(2, 4)
        
        # Calculate amounts - explicitly cast to float
        unit_price = float(product["UnitPrice"])
        unit_cost = float(product["UnitCost"])
        
        # Discount (more likely in certain months)
        if txn_date.month in [1, 2, 8, 9]:
            discount_prob = 0.35
        elif txn_date.month in [11, 12]:
            discount_prob = 0.25
        else:
            discount_prob = 0.15
        
        if random.random() < discount_prob:
            discount = round2(unit_price * random.choice([0.10, 0.15, 0.20, 0.25]))
        else:
            discount = 0.0
        
        total_amount = round2((unit_price - discount) * qty)
        total_cost = round2(unit_cost * qty)
        
        # Customer ID (70% loyalty members)
        customer_id = f"C{random.randint(10001, 60000)}" if random.random() < 0.7 else ""
        
        transactions.append((
            str(txn_id),
            str(txn_date.strftime("%Y-%m-%d")),
            str(store_id),
            str(product["ProductID"]),
            str(customer_id),
            int(qty),
            float(unit_price),
            float(discount),
            float(total_amount),
            float(total_cost)
        ))
    
    return transactions

print(f"Generating {NUM_TRANSACTIONS:,} transactions...")
sales_data = generate_transactions(NUM_TRANSACTIONS)
print(f"Generated {len(sales_data):,} valid transactions")

In [ ]:
# Create Sales DataFrame
sales_schema = StructType([
    StructField("TransactionID", StringType(), False),
    StructField("TransactionDate", StringType(), True),
    StructField("StoreID", StringType(), True),
    StructField("ProductID", StringType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("Discount", DoubleType(), True),
    StructField("TotalAmount", DoubleType(), True),
    StructField("TotalCost", DoubleType(), True),
])

df_sales = spark.createDataFrame(sales_data, schema=sales_schema)

# Use SQL-style cast to avoid col() function issues
df_sales.createOrReplaceTempView("sales_temp")
df_sales = spark.sql("""
    SELECT 
        TransactionID,
        TO_DATE(TransactionDate) AS TransactionDate,
        StoreID,
        ProductID,
        CustomerID,
        Quantity,
        UnitPrice,
        Discount,
        TotalAmount,
        TotalCost
    FROM sales_temp
""")

print(f"Sales: {df_sales.count():,} rows")
df_sales.show(5)

## 6. Data Summary

Quick validation of generated data.

In [ ]:
# Summary statistics using SQL to avoid function conflicts
print("=" * 50)
print("DATA SUMMARY")
print("=" * 50)

summary = spark.sql("""
    SELECT 
        COUNT(*) as total_txn,
        SUM(TotalAmount) as total_revenue,
        SUM(TotalCost) as total_cost
    FROM sales_temp
""").collect()[0]

total_txn = summary["total_txn"]
total_revenue = summary["total_revenue"]
total_cost = summary["total_cost"]

print(f"\nTransactions: {total_txn:,}")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Gross Margin: {(total_revenue - total_cost) / total_revenue * 100:.1f}%")
print(f"Avg Transaction: ${total_revenue / total_txn:.2f}")

print("\n" + "=" * 50)
print("REVENUE BY YEAR")
print("=" * 50)

spark.sql("""
    SELECT 
        YEAR(TransactionDate) AS Year,
        SUM(TotalAmount) AS Revenue
    FROM sales_temp
    GROUP BY YEAR(TransactionDate)
    ORDER BY Year
""").show()

## 7. Save to Lakehouse

Write all tables as Delta format to the Lakehouse.

In [ ]:
# Save mode: "overwrite" for fresh load, "append" for incremental
mode = "append" if APPEND_MODE else "overwrite"

print(f"Saving tables (mode: {mode})...")

# Dimension tables (always overwrite)
df_products.write.format("delta").mode("overwrite").saveAsTable("Products")
print("✓ Products")

df_stores.write.format("delta").mode("overwrite").saveAsTable("Stores")
print("✓ Stores")

df_calendar.write.format("delta").mode("overwrite").saveAsTable("Calendar")
print("✓ Calendar")

df_targets.write.format("delta").mode("overwrite").saveAsTable("Targets")
print("✓ Targets")

# Fact table (can append)
df_sales.write.format("delta").mode(mode).saveAsTable("Sales")
print(f"✓ Sales ({mode})")

print("\n" + "=" * 50)
print("ALL TABLES SAVED TO LAKEHOUSE!")
print("=" * 50)

---

## 🎯 Demo: Add Incremental Data

Run this section during a demo to show data changes flowing through.

**Use Case:** Show Git detecting changes after new data is added.

In [ ]:
# === DEMO: ADD NEW TRANSACTIONS ===
# Uncomment and run during demo to add fresh data

# NEW_TRANSACTIONS = 1000  # Add 1000 new transactions
# START_ID = 2000001  # Start from a new ID range
# 
# print(f"Adding {NEW_TRANSACTIONS} new transactions...")
# new_sales_data = generate_transactions(NEW_TRANSACTIONS, start_id=START_ID)
# df_new_sales = spark.createDataFrame(new_sales_data, sales_schema)
# df_new_sales = df_new_sales.withColumn("TransactionDate", to_date(col("TransactionDate")))
# 
# df_new_sales.write.format("delta").mode("append").saveAsTable("Sales")
# print(f"✓ Added {df_new_sales.count()} new transactions!")
# 
# # Verify
# total = spark.sql("SELECT COUNT(*) FROM Sales").collect()[0][0]
# print(f"Total transactions now: {total:,}")

---

## Next Steps

1. **Create a Semantic Model** pointing to these Delta tables
2. **Connect to Git** from the Fabric workspace
3. **Build reports** using the Sales Analytics model

See the workshop materials for detailed instructions.